In [1]:
import pickle
import numpy as np
import tensorflow as tf


/home/snu/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data

In [3]:
#load the CIFAR-10
def load_CIFAR10(pos, n_chunks=1):
    Xtr = []
    Ytr = []
    for i in range(n_chunks): #batch_1 ~ batch_5에 대해 : batch 1개당 10000개의 그림이다.
        train = unpickle(pos + '/data_batch_{0}'.format(i + 1)) #경로/data_batch
        Xtr.extend(train[b'data'])
        Ytr.extend(train[b'labels'])
        test = unpickle(pos + '/test_batch')
        Xte = test[b'data']
        Yte = test[b'labels']
    return np.array(Xtr), np.array(Ytr), np.array(Xte), np.array(Yte)

train=unpickle('cifar-10-batches-py/data_batch_1')
xtrain=np.array(train[b'data'])
xtrain.shape

(10000, 3072)

In [4]:
# expresses the label data in one-hot encoding.
def onehot_encoding (Ytr, Yte):
    Ytr_onehot = np.zeros((Ytr.size, 10))
    Yte_onehot = np.zeros((Yte.size, 10))
    for i in range(Ytr.size):
        Ytr_onehot[i][Ytr[i]] = 1
    for i in range(Yte.size):
        Yte_onehot[i][Yte[i]] = 1
    return Ytr_onehot, Yte_onehot

In [5]:
# load the train and test data
Xtr, Ytr, Xte, Yte = load_CIFAR10('cifar-10-batches-py', 5)
                                 
# image data, each data size is 32*32*3
Xtr = Xtr.reshape(50000, 3, 32, 32).transpose(0,2,3,1).astype("float")
#50000 32 32 3

Xte= Xte.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
#10000 32 32 3

# label data of train and test data, label data is represented by one-hot encoding
Ytr_onehot, Yte_onehot = onehot_encoding(Ytr, Yte)
#50000 10
#10000 10

Xtr.shape
Ytr_onehot.shape

(50000, 10)

In [6]:
### placeholder 생성 (X변수, Y변수)

X = tf.placeholder(tf.float32, [None, 32, 32, 3])
#None: 이미지판의 개수
#32: 판의 세로 길이. 즉 행의 수
#32: 판의 가로 길이. 즉 열의 수
#3: 가장 안쪽 내용물 []의 차원 수.  몇개의 숫자를 보유하는가?

Y = tf.placeholder(tf.float32, [None, 10])


#dropout_prob = tf.placeholder(tf.float32)

In [7]:
### Implement the layers of CNNs ###

### Variable 생성 (W1W2....) - 랜덤으로 돌려서 구해야 하는 것
W1 = tf.Variable(tf.random_normal([3,3,3,32], stddev=0.01)) 
# padding = 'SAME' keeps output size equal to input.
# 첫번째 conv layer의 w1이며, 추후에 랜덤으로 생성할 것이다
# 3: 필터의 세로 길이
# 3: 필터의 가로 길이
# 1: 가장 안쪽 내용물의 차원수 위의 4번째와 같다
# 32: 필터의 개수

#1번-convolution
L1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding='SAME')
#tf.nn.conv2d(input하는X, weight, strides, padding)
#strides=[이미지는1로고정. 세로는1씩이동. 가로는 1씩이동. 필터는1로고정]
#same: 필터를 거치고 나서 사이즈를 같게 유지싴켜줌
print(L1)

#1번-relu
L1 = tf.nn.relu(L1)
print(L1)
# Pooling layer

#1번-pooling
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
# tf.nn.max_pool(L1,)
# 1로 고정(?) 아마...
# 2*2 풀링필터는 2x2 사이즈이다
# 풀링필터의 개수는 1겹
# ksize풀링필터의 사이즈 = 1 2 2 1
# strides=1 2 2 1 (2칸씩 움직임)  -> 32*32 (32층) -> 16*16 (32층)으로 줄어든다
# padding=same
print(L1)


#->최종 결과: ?, 16, 16, 32

Tensor("Conv2D:0", shape=(?, 32, 32, 32), dtype=float32)
Tensor("Relu:0", shape=(?, 32, 32, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 16, 16, 32), dtype=float32)


In [8]:
### 2번 Layer

# Variable
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
# 3*3 필터
# 32: 아까 32층 필터이므로
# 새필터는 64층 필터

# convolution
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
print(L2)

L2 = tf.nn.relu(L2)
print(L2)

L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
print(L2)

Tensor("Conv2D_1:0", shape=(?, 16, 16, 64), dtype=float32)
Tensor("Relu_1:0", shape=(?, 16, 16, 64), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 8, 8, 64), dtype=float32)


In [9]:
### 3번 Layer: 2차원으로 통합해준다

W3 = tf.Variable(tf.random_normal([8*8*64, 256], stddev=0.01))
#1개의 이미지당 256개 수로 도출시켜주는 weight이다 

L3 = tf.reshape(L2, [-1, 8*8*64])
#위에서 L2의 사이즈가 8*8*64이므로
print(L3)


L3 = tf.matmul(L3, W3)
# ? 8864 X 8864 256
print(L3)

L3 = tf.nn.relu(L3)
print(L3)

#L3 = tf.nn.dropout(L3, dropout_prob)
#print(L3)

Tensor("Reshape:0", shape=(?, 4096), dtype=float32)
Tensor("MatMul:0", shape=(?, 256), dtype=float32)
Tensor("Relu_2:0", shape=(?, 256), dtype=float32)


In [10]:
### 4번 Layer: Fully connected layer

W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
# 1의 이미지당 10개 라벨로 도출해주는 weight이다
print(W4)

model = tf.matmul(L3, W4)
# ? 256 X 256 10 
print(model)

#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
#optimizer = tf.train.AdamOptimizer(0.005).minimize(cost)

#init = tf.global_variables_initializer()
#sess = tf.Session()
#sess.run(init) 

<tf.Variable 'Variable_3:0' shape=(256, 10) dtype=float32_ref>
Tensor("MatMul_1:0", shape=(?, 10), dtype=float32)


In [11]:
# cost function, you can change the implementation
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.01).minimize(cost)

In [12]:
# initialize the variables
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [13]:
### Implement the train process ###




In [14]:
### Implement the test process ###

In [15]:
batch_size = 2000
total_batch = int(50000 / batch_size)
total_batch

25

In [16]:
Xtr[0].shape

(32, 32, 3)

In [17]:
Xtr[:10000].shape
Xtr[10000:20000].shape
Ytr_onehot[:10000].shape
Ytr_onehot[10000:20000].shape

(10000, 10)

In [18]:

for epoch in range(10):
    total_cost = 0
    
    for i in range(total_batch): #total_batch: 나눠진 등분의 개수
        batch_xs=Xtr[batch_size*(i):batch_size*(i+1)] #batch_size: 한등분의 그림의 개수
        batch_ys=Ytr_onehot[batch_size*(i):batch_size*(i+1)] 
        _, curr_loss = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs,
                                          Y: batch_ys})
        total_cost += curr_loss
        
    print('Epoch:', '%04d' % (epoch +1),
         'Avg. cost = ', '{:,.3f}'.format(total_cost/total_batch))

Epoch: 0001 Avg. cost =  8.112
Epoch: 0002 Avg. cost =  2.303
Epoch: 0003 Avg. cost =  2.303
Epoch: 0004 Avg. cost =  2.303
Epoch: 0005 Avg. cost =  2.303


KeyboardInterrupt: 

In [19]:
correctness = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32))
print('Accuracy', sess.run(accuracy, 
                           feed_dict={
                               X:Xte,
                               Y: Yte_onehot}))

Accuracy 0.4913
